In [1]:
!export TOKENIZERS_PARALLELISM=false

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name="jhgan/ko-sbert-nli",
    normalize=True,
)

Settings.llm = OpenAI(temperature=0, model="gpt-4")
Settings.embed_model = embed_model

In [5]:
from llama_index.readers.file import PyMuPDFReader

visa_pdf_file_path = "../../../Documents/E-9 Visa Guide_한국어.pdf"
visa_docs = PyMuPDFReader().load_data(file_path=visa_pdf_file_path, metadata=True)

In [9]:
import json

metadata_json_path = "../../../Documents/E-9_Visa_Guide_한국어_metadata.json"
with open(metadata_json_path, "r") as fp:
    metadata_json = json.load(fp)

print(metadata_json[0])

{'page_label': '1', 'file_name': 'E-9 Visa Guide_한국어2.pdf', 'file_path': 'data/visa_docs/E-9 Visa Guide_한국어2.pdf', 'file_type': 'application/pdf', 'file_size': 123849, 'creation_date': '2024-03-22', 'last_modified_date': '2024-03-22', 'document_title': 'E-9 Visa Guide: 목차', 'questions_this_excerpt_can_answer': '1. E-9 비자 가이드에 따르면, 고용허가제 해당자가 사증을 발급받기 위해 필요한 절차는 무엇인가요?\n2. 고용허가제 해당자가 근무처를 변경할 수 있는 조건과 그 절차는 어떻게 되나요?', 'section_summary': '이 문서는 E-9 비자 가이드에 대한 내용을 담고 있습니다. 고용허가제 해당자의 사증 발급 절차, 허용 업종 및 체류 자격, 사증 발급 인정서 발급 절차 등에 대해 설명하고 있습니다. 또한, 고용허가제 해당자가 근무처를 변경할 수 있는 조건과 그 절차, 필요한 제출 서류 등에 대한 정보도 포함하고 있습니다. 이 외에도 고용허가제 해당자의 체류자격 변경 허가, 체류 기간 연장 허가, 재입국 허가, 외국인 등록, 고용 변동 신고 등에 대한 내용도 다루고 있습니다.', 'excerpt_keywords': 'E-9 비자, 고용허가제, 사증 발급, 근무처 변경, 체류자격 변경'}


In [12]:
dict1 = {"a": "a", "c": 'ca'}
dict2 = {"b": "b", "c": 'cb'}
dict3 = {**dict1, **dict2}

dict3

{'a': 'a', 'c': 'cb', 'b': 'b'}

In [14]:
from llama_index.core.schema import TextNode

nodes = [TextNode(text=doc.text, metadata={**metadata_json[idx], **doc.metadata}) for idx, doc in enumerate(visa_docs)]

In [19]:
nodes[0].metadata

{'page_label': '1',
 'file_name': 'E-9 Visa Guide_한국어2.pdf',
 'file_path': '../../../Documents/E-9 Visa Guide_한국어.pdf',
 'file_type': 'application/pdf',
 'file_size': 123849,
 'creation_date': '2024-03-22',
 'last_modified_date': '2024-03-22',
 'document_title': 'E-9 Visa Guide: 목차',
 'questions_this_excerpt_can_answer': '1. E-9 비자 가이드에 따르면, 고용허가제 해당자가 사증을 발급받기 위해 필요한 절차는 무엇인가요?\n2. 고용허가제 해당자가 근무처를 변경할 수 있는 조건과 그 절차는 어떻게 되나요?',
 'section_summary': '이 문서는 E-9 비자 가이드에 대한 내용을 담고 있습니다. 고용허가제 해당자의 사증 발급 절차, 허용 업종 및 체류 자격, 사증 발급 인정서 발급 절차 등에 대해 설명하고 있습니다. 또한, 고용허가제 해당자가 근무처를 변경할 수 있는 조건과 그 절차, 필요한 제출 서류 등에 대한 정보도 포함하고 있습니다. 이 외에도 고용허가제 해당자의 체류자격 변경 허가, 체류 기간 연장 허가, 재입국 허가, 외국인 등록, 고용 변동 신고 등에 대한 내용도 다루고 있습니다.',
 'excerpt_keywords': 'E-9 비자, 고용허가제, 사증 발급, 근무처 변경, 체류자격 변경',
 'total_pages': 21,
 'source': '1'}

In [20]:
# nodes[5]

print(len(nodes))

21


In [ ]:
for node in nodes:
    page = node.metadata['source']
    print(f"Page: {page}\nContent:\n{node.get_content()}\n\n----------------------------------------------------------------\n\n")

In [7]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter
import os
from tqdm.notebook import tqdm
import pickle
from pathlib import Path

llm = OpenAI(model="gpt-4")

async def build_agent_per_page(nodes, file_base):
    print(file_base)

    vi_out_path = f"./data/E9_VISA/{file_base}"
    summary_out_path = f"./data/E9_VISA/{file_base}_summary.pkl"
    if not os.path.exists(vi_out_path):
        Path("./data/E9_VISA/").mkdir(parents=True, exist_ok=True)
        # build vector index
        vector_index = VectorStoreIndex(nodes)
        vector_index.storage_context.persist(persist_dir=vi_out_path)
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=vi_out_path),
        )

    # build summary index
    summary_index = SummaryIndex(nodes)

    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=llm)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize", llm=llm
    )

    # extract a summary
    if not os.path.exists(summary_out_path):
        Path(summary_out_path).parent.mkdir(parents=True, exist_ok=True)
        summary = str(
            await summary_query_engine.aquery(
                "Extract a concise 1-2 line summary of this document"
            )
        )
        pickle.dump(summary, open(summary_out_path, "wb"))
    else:
        summary = pickle.load(open(summary_out_path, "rb"))

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"vector_tool_{file_base}",
                description=f"Useful for questions related to specific facts",
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name=f"summary_tool_{file_base}",
                description=f"Useful for summarization questions",
            ),
        ),
    ]

    # build agent
    function_llm = OpenAI(model="gpt-4")
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""\
You are a specialized agent designed to answer queries about the `{page_title}` part of E-9 Visa Guide_한국어 Document.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
    )

    return agent

In [ ]:
"""
agents_dict:
{
    [page_title]: Agent
}

extra_info_dict: 
{
    [page_title]: {"summary": summary, "nodes": nodes}
}
"""


agents_dict = {}
extra_info_dict = {}


for idx, node in enumerate(nodes):
    metadata = metadata_json[idx]

    page_label = metadata["page_label"]
    page_title = metadata["document_title"]
    section_summary = metadata["section_summary"]

    key = f"E9_Visa_Guide_{page_label}"

    extra_info_dict[key] = {"title": page_title, "summary": section_summary, "nodes": [node]}

    agent = await build_agent_per_page(nodes=[node], file_base=key, page_title=page_title)
    agents_dict[key] = agent

In [24]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# define tool for each document agent
all_tools = []
for key, agent in agents_dict.items():
    summary = extra_info_dict[key]["summary"]
    doc_tool = QueryEngineTool(
        query_engine=agent,
        metadata=ToolMetadata(
            name=f"tool_{key}",
            description=summary,
        ),
    )
    all_tools.append(doc_tool)

In [ ]:
print(all_tools[10].metadata)
print(extra_info_dict["E9_Visa_Guide:11"])

In [25]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import (
    ObjectIndex,
    SimpleToolNodeMapping,
    ObjectRetriever,
)
from llama_index.core.retrievers import BaseRetriever
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4-0613")

tool_mapping = SimpleToolNodeMapping.from_objects(all_tools)
obj_index = ObjectIndex.from_objects(
    all_tools,
    tool_mapping,
    VectorStoreIndex,
)
vector_node_retriever = obj_index.as_node_retriever(similarity_top_k=10)


# define a custom retriever with reranking
class CustomRetriever(BaseRetriever):
    def __init__(self, vector_retriever, postprocessor=None):
        self._vector_retriever = vector_retriever
        self._postprocessor = postprocessor or CohereRerank(top_n=5)
        # self._postprocessor = postprocessor
        super().__init__()

    def _retrieve(self, query_bundle):
        retrieved_nodes = self._vector_retriever.retrieve(query_bundle)
        filtered_nodes = self._postprocessor.postprocess_nodes(
            retrieved_nodes, query_bundle=query_bundle
        )

        return filtered_nodes


# define a custom object retriever that adds in a query planning tool
class CustomObjectRetriever(ObjectRetriever):
    def __init__(self, retriever, object_node_mapping, all_tools, llm=None):
        self._retriever = retriever
        self._object_node_mapping = object_node_mapping
        self._llm = llm or OpenAI("gpt-4-0613")

    def retrieve(self, query_bundle):
        nodes = self._retriever.retrieve(query_bundle)
        tools = [self._object_node_mapping.from_node(n.node) for n in nodes]

        sub_question_engine = SubQuestionQueryEngine.from_defaults(
            query_engine_tools=tools, llm=self._llm
        )
        sub_question_description = f"""\
Useful for any queries that involve comparing multiple documents. ALWAYS use this tool for comparison queries - make sure to call this \
tool with the original query. Do NOT use the other tools for any queries involving multiple documents.
"""
        sub_question_tool = QueryEngineTool(
            query_engine=sub_question_engine,
            metadata=ToolMetadata(
                name="compare_tool", description=sub_question_description
            ),
        )

        return tools + [sub_question_tool]

In [26]:
from llama_index.core.postprocessor import SimilarityPostprocessor

# CohereRerank를 대신해서...
# postprocessor = SimilarityPostprocessor(similarity_cutoff=0.4)

custom_node_retriever = CustomRetriever(vector_node_retriever)

# wrap it with ObjectRetriever to return objects
custom_obj_retriever = CustomObjectRetriever(
    custom_node_retriever, tool_mapping, all_tools, llm=llm
)

In [27]:
tmps = custom_obj_retriever.retrieve("비전문취업 비자의 일반적인 허용 업종 범위는 어디까지인가요?")
print(len(tmps))

6


In [28]:
from llama_index.agent.openai_legacy import FnRetrieverOpenAIAgent
from llama_index.core.agent import ReActAgent

top_agent = FnRetrieverOpenAIAgent.from_retriever(
    custom_obj_retriever,
    system_prompt=""" \
You are an agent designed to answer queries about the documentation.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    llm=llm,
    verbose=True,
)

# top_agent = ReActAgent.from_tools(
#     tool_retriever=custom_obj_retriever,
#     system_prompt=""" \
# You are an agent designed to answer queries about the documentation.
# Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

# """,
#     llm=llm,
#     verbose=True,
# )

In [17]:
def pretty_source_nodes(response):
    for node in response.source_nodes:
        print(node)

In [18]:
def save_llm_result(question, base_txt, agent_txt):
    # 파일을 추가 모드('a')로 열기
    with open('평가:E-9 Visa.md', 'a') as file:
        # 파일에 텍스트 추가
        file.write((
            f"\n\n------------------------------------------------------------------------------------------------------------------------------------------"
            f"\n\n### Q. {question}"
            f"\n##### Base:"
            f"\n{base_txt}"
            f"\n\n##### Agent:"
            f"\n{agent_txt}"
        ))

In [29]:
# all_nodes = [
#     extra_info["node"] for extra_info in extra_info_dict.values()
# ]

all_nodes = [
    n for extra_info in extra_info_dict.values() for n in extra_info["nodes"]
]

base_index = VectorStoreIndex(all_nodes)
base_query_engine = base_index.as_query_engine(similarity_top_k=4)

In [32]:
questions = [
    # "E-9 비자 신청 시 필요한 서류는 무엇인가요?",
    # "고용허가서 신청 절차는 어떻게 되나요?",
    # "재입국 특례 제도 신청을 위한 조건은 무엇인가요?",
    "비전문취업 비자의 일반적인 허용 업종 범위는 어디까지인가요?",
]

In [33]:
for question in questions:
    base_response = base_query_engine.query(question)
    agent_response = top_agent.query(question)
    save_llm_result(question, str(base_response), str(agent_response))
    pretty_source_nodes(agent_response)
    print(f"\n\n\n")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: tool_E9_Visa_Guide_5 with args: {
  "input": "비전문취업 비자의 일반적인 허용 업종 범위"
}
Added user message to memory: 비전문취업 비자의 일반적인 허용 업종 범위
=== Calling Function ===
Calling function: vector_tool_E9_Visa_Guide_5 with args: {
  "input": "비전문취업 비자의 일반적인 허용 업종 범위"
}
Got output: 비전문취업 비자(E-9)의 허용 업종 범위는 다음과 같습니다:

1. 제조업: 상시 근로자 300인 미만 또는 자본금 80억원 이하의 제조업체에서 근로자를 고용할 수 있습니다.
2. 건설업: 모든 건설공사에서 근로자를 고용할 수 있습니다. 단, 발전소, 제철소, 석유화학 건설 현장의 건설업체 중 건설면허가 산업환경설비인 경우는 적용이 제외됩니다.
3. 농축산업: 작물재배업, 축산업, 농축산서비스업에서 근로자를 고용할 수 있습니다.
4. 어업: 연안어업, 근해어업, 양식어업, 소금채취업에서 근로자를 고용할 수 있습니다.
5. 임업: 임업종묘생산업, 육림업, 벌목업, 임업관련서비스업에서 근로자를 고용할 수 있습니다.
6. 광업: 금속광업, 비금속광물광업에서 근로자를 고용할 수 있습니다.
7. 서비스업: 건설폐기업, 냉장냉동업, 호텔숙박업, 음식점업, 재료수집업, 출판업 등에서 근로자를 고용할 수 있습니다. 

각 업종에는 특정 조건이 적용되며, 이는 해당 업종에서 근로자를 고용할 수 있는 조건을 나타냅니다.

Got output: 비전문취업 비자(E-9)의 허용 업종 범위는 다음과 같습니다:

1. 제조업: 상시 근로자 300인 미만 또는 자본금 80억원 이하의 제조업체에서 근로자를 고용할 수 있습니다.
2. 건설업: 모든 건설공사에서 근로자를 고용할 수 있습니다. 단,